## AI 모델 with Counter, Synerge

데이터 추출

In [16]:
VERSION ='10.23'
TEAM = 4
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import boto3
from boto3.dynamodb.conditions import Key, Attr
from keras.models import load_model
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import optimizers
import numpy as np
from keras.models import load_model
from sklearn.model_selection import KFold, cross_val_score

In [17]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('matchData')

scan_kwargs={
    'FilterExpression': Attr('version').eq(VERSION),
}
done= False
start_key =None
data=[]
while not done:
    if start_key:
        scan_kwargs['ExclusiveStartKey'] = start_key
    response = table.scan(**scan_kwargs)
    temp = response.get('Items', [])
    data = data+temp
    start_key = response.get('LastEvaluatedKey', None)
    done = start_key is None

len(data)

11200

In [40]:
df = pd.DataFrame(data)
df.info()
df.sample(2)
df['players']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11200 entries, 0 to 11199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   matchID  11200 non-null  object
 1   version  11200 non-null  object
 2   players  11200 non-null  object
 3   ban      11200 non-null  object
dtypes: object(4)
memory usage: 350.1+ KB


0        [{'pickturn': 1, 'championID': 202, 'championN...
1        [{'pickturn': 1, 'championID': 876, 'championN...
2        [{'pickturn': 1, 'championID': 164, 'championN...
3        [{'pickturn': 1, 'championID': 876, 'championN...
4        [{'pickturn': 1, 'championID': 114, 'championN...
                               ...                        
11195    [{'pickturn': 1, 'championID': 30, 'championNa...
11196    [{'pickturn': 1, 'championID': 76, 'championNa...
11197    [{'pickturn': 1, 'championID': 41, 'championNa...
11198    [{'pickturn': 1, 'championID': 202, 'championN...
11199    [{'pickturn': 1, 'championID': 163, 'championN...
Name: players, Length: 11200, dtype: object

In [41]:
df['players'].to_json('./Data/DataFrame/train_data.json',orient='table')

In [42]:
my=pd.read_json('./Data/DataFrame/train_data.json')
my

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

Synerge Counter Pre-Processing

In [19]:
synerge = dict()
counter = dict()
winRate = dict()
champ_ID = pd.read_csv("./Data/championName_ID.csv")
for champ in champ_ID['name']:
    if champ == "No ban":
        continue
    synerge[champ]=dict()
    counter[champ]=dict()
    winRate[champ]={'Win':0,"Lose":0,"WinRate":0}
    for champ2 in champ_ID['name']:
        if champ == "No ban":
            continue
        if(champ!=champ2):
            synerge[champ][champ2]={'Win':0,"Lose":0,"WinRate":0}
            counter[champ][champ2]={'Win':0,"Lose":0,"WinRate":0}

print('End')

End


In [20]:
for players in df['players']:
    #red team
    redWin = True
    blueWin = False
    if players[0]['win']==True:
        redWin = True
        blueWin = False
    else:
        redWin = True
        blueWin = False

#Synerge
    for i in range(TEAM):
        for j in range(TEAM):
            if(i!=j):
                if(redWin):
                    synerge[players[i]['championName']][players[j]['championName']]['Win'] +=1
                else:
                    synerge[players[i]['championName']][players[j]['championName']]['Lose'] +=1
    for i in range(TEAM):
        i=i+5
        for j in range(TEAM):
            j=j+5
            if(i!=j):
                if(blueWin):
                    synerge[players[i]['championName']][players[j]['championName']]['Win'] +=1
                else:
                    synerge[players[i]['championName']][players[j]['championName']]['Lose'] +=1

#Counter
    for i in range(TEAM):
        for j in range(5):
            j=j+TEAM
            if(redWin):
                counter[players[i]['championName']][players[j]['championName']]['Win'] +=1
            else:
                counter[players[i]['championName']][players[j]['championName']]['Lose'] +=1
    for i in range(TEAM):
        i=i+5
        for j in range(TEAM):
            if(blueWin):
                counter[players[i]['championName']][players[j]['championName']]['Win'] +=1
            else:
                counter[players[i]['championName']][players[j]['championName']]['Lose'] +=1

#Winrate
    for i in range(10):
        if(redWin):
            if(i<5):
                winRate[players[i]['championName']]['Win'] +=1
            else:
                winRate[players[i]['championName']]['Lose'] +=1
        else:
            if(i<5):
                winRate[players[i]['championName']]['Lose'] +=1
            else:
                winRate[players[i]['championName']]['Win'] +=1
print('End')

End


Win rate calculate

In [21]:
for champ in champ_ID['name']:
    if champ =="No ban":
        continue
    if winRate[champ]['Win']+winRate[champ]['Lose']>0:
        winRate[champ]['WinRate']=(100*winRate[champ]['Win'])/(winRate[champ]['Win']+winRate[champ]['Lose'])
        winRate[champ]['WinRate']-=50
    for champ2 in champ_ID['name']:
        if champ2 == "No ban":
            continue
        if(champ==champ2):
            continue
        mySynerge = synerge[champ][champ2]
        if mySynerge['Win']+mySynerge['Lose'] > 0:
            mySynerge['WinRate']=100*mySynerge['Win']/(mySynerge['Win']+mySynerge['Lose'])
        else:
            mySynerge['WinRate']=50
        myCounter = counter[champ][champ2]
        if myCounter['Win']+myCounter['Lose'] > 0:
            myCounter['WinRate']=100*myCounter['Win']/(myCounter['Win']+myCounter['Lose'])
        else:
            myCounter["WinRate"]=50
print(counter["Ornn"]["Camille"])
print(winRate["Ornn"]['WinRate'])

{'Win': 27, 'Lose': 38, 'WinRate': 41.53846153846154}
0.26595744680851396


Input data Pre-Processing

In [31]:
myData=pd.DataFrame()
print(myData)
for players in df['players']:
    isError = False
    try:
        champList=[]
        for player in players:
            champList.append(player['championName'])

        if(players[0]['win']==True):
            win0=1
            win1=0
        else:
            win0=0
            win1=1
        champList.append(win0)

        InputList=[]

        #Red

        for i in range(TEAM):
            champ = champList[i]
            myS=0
            myC=0
            for j in range(TEAM):
                champ2 = champList[j]
                if champ != champ2:
                    myS += synerge[champ][champ2]["WinRate"]
                champ2 = champList[j+5]
                myC += counter[champ][champ2]["WinRate"]
            if TEAM>1:
                myS = myS/TEAM-2
            myC = myC/TEAM
            InputList.append(winRate[champ]['WinRate'])
            InputList.append(myS-50)
            InputList.append(myC-50)

        #Blue
        for i in range(TEAM):
            i=i+5
            champ = champList[i]
            myS=0
            myC=0
            for j in range(TEAM):
                champ2 = champList[j+5]
                if champ != champ2:
                    myS += synerge[champ][champ2]["WinRate"]
                champ2 = champList[j]
                myC += counter[champ][champ2]["WinRate"]
            if TEAM>1:
                myS = myS/TEAM-1
            myC = myC/TEAM
            InputList.append(winRate[champ]['WinRate'])
            InputList.append(myS-50)
            InputList.append(myC-50)

        Columns=[]
        for i in range(TEAM):
            Columns.append("W"+(str)(i+1))
            Columns.append("S"+(str)(i+1))
            Columns.append("C"+(str)(i+1))
        for i in range(TEAM):
            Columns.append("W"+(str)(i+6))
            Columns.append("S"+(str)(i+6))
            Columns.append("C"+(str)(i+6))
        Columns.append("win")
        InputList.append(champList[10])
        champList = pd.DataFrame([InputList],
                                 columns=Columns)
    except Exception as e:
        print(e)
        isError = True
    if(isError):
        continue
    myData=myData.append(champList)
    myData=myData.reset_index(drop=True)

myData.info()

Empty DataFrame
Columns: []
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11200 entries, 0 to 11199
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   W1      11200 non-null  float64
 1   S1      11200 non-null  float64
 2   C1      11200 non-null  float64
 3   W2      11200 non-null  float64
 4   S2      11200 non-null  float64
 5   C2      11200 non-null  float64
 6   W3      11200 non-null  float64
 7   S3      11200 non-null  float64
 8   C3      11200 non-null  float64
 9   W4      11200 non-null  float64
 10  S4      11200 non-null  float64
 11  C4      11200 non-null  float64
 12  W6      11200 non-null  float64
 13  S6      11200 non-null  float64
 14  C6      11200 non-null  float64
 15  W7      11200 non-null  float64
 16  S7      11200 non-null  float64
 17  C7      11200 non-null  float64
 18  W8      11200 non-null  float64
 19  S8      11200 non-null  float64
 20  C8      11200 non-null  float64
 2

In [32]:
x= myData[Columns[:-1]]
y=myData[Columns[-1]]
x_train_all,x_test,y_train_all,y_test = tts(x,y,train_size=0.8)
x_train,x_valid,y_train,y_valid=tts(x_train_all,y_train_all,train_size=0.8)

x_train_all = x_train_all.to_numpy(dtype='float32')
y_train_all = y_train_all.to_numpy(dtype='float32')

x_train = x_train.to_numpy(dtype='float32')
y_train = y_train.to_numpy(dtype='float32')

x_valid = x_valid.to_numpy(dtype='float32')
y_valid = y_valid.to_numpy(dtype='float32')

x_test = x_test.to_numpy(dtype='float32')
y_test = y_test.to_numpy(dtype='float32')

print("end")

end


In [33]:
from keras import regularizers
def build_model():
    model = Sequential()
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    #Compile model
    adam = optimizers.Adam(lr = 0.001)
    Nadam = optimizers.Nadam()
    model.compile(loss='binary_crossentropy', optimizer=Nadam, metrics=['accuracy'])

    return model
print("end")

end


In [34]:
from sklearn.model_selection import StratifiedKFold
model = build_model()
skf = StratifiedKFold(n_splits=10)
i=0
'''
for train_idx,test_idx in skf.split(x_train_all,y_train_all):
    i+=1
    print(i)
    x_train, y_train = x_train_all[train_idx],y_train_all[train_idx]
'''
history = model.fit(x_train, y_train,validation_split=0.1, epochs=200,batch_size=32, verbose=1,shuffle=True)

Epoch 1/200
202/202 [==============================] - 1s 3ms/step - loss: 1.1913 - accuracy: 0.5086 - val_loss: 0.7078 - val_accuracy: 0.4895
Epoch 2/200
202/202 [==============================] - 0s 2ms/step - loss: 0.7637 - accuracy: 0.4937 - val_loss: 0.6951 - val_accuracy: 0.5160

KeyboardInterrupt: 

In [26]:
def score(model,x_test,y_test,param):
    same=0
    result = model.predict(x_test)
    count=0
    for i in range(len(result)):
        if result[i]>param:
            predict=1
        elif result[i]<1-param:
            predict=0
        else:
            continue
        count=count+1
        if(predict==y_test[i]):
            same+=1
        else:
            #print(result[i])
            pass
    acc = same/count
    print(acc,count)
    return acc

In [35]:
score(model,x_test[:1000],y_test[:1000],0.5)
score(model,x_test[:1000],y_test[:1000],0.6)
score(model,x_test[:1000],y_test[:1000],0.7)
score(model,x_test[:1000],y_test[:1000],0.8)
score(model,x_test[:1000],y_test[:1000],0.9)

0.534 1000
0.5523255813953488 172
0.5545454545454546 110
0.5333333333333333 75
0.5370370370370371 54


0.5370370370370371

In [28]:
model.save('./Y_model_201204_B.h5')

In [29]:
mymodel = load_model('Y_model_201204_90_60.h5')
score(mymodel,x_test[:1000],y_test[:1000],0.5)
score(mymodel,x_test[:1000],y_test[:1000],0.6)
score(mymodel,x_test[:1000],y_test[:1000],0.7)
score(mymodel,x_test[:1000],y_test[:1000],0.8)
score(mymodel,x_test[:1000],y_test[:1000],0.9)

ValueError: in user code:

    C:\Users\aranz\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\aranz\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\aranz\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\aranz\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\aranz\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\aranz\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run_step 

In [44]:
import pandas as pd
def Convert(champList):
    TEAM = len(champList)
    synerge = pd.read_json('./Data/WSC/Synerge'+(str)(TEAM)+'.json')
    counter = pd.read_json('./Data/WSC/Counter'+(str)(TEAM)+'.json')
    winRate = pd.read_json('./Data/WSC/WinRate'+(str)(TEAM)+'.json')
    InputList=[]

    TEAM = len(champList)/2

    #Red
    for i in range(TEAM):
        champ = champList[i]
        myS=0
        myC=0
        for j in range(TEAM):
            champ2 = champList[j]
            if champ != champ2:
                myS += synerge[champ][champ2]["WinRate"]
            champ2 = champList[j+TEAM]
            myC += counter[champ][champ2]["WinRate"]
        if TEAM>1:
            myS = myS/(TEAM-1)
        myC = myC/TEAM
        InputList.append(winRate[champ]['WinRate'])
        InputList.append(myS-50)
        InputList.append(myC-50)

    #Blue
    for i in range(TEAM):
        i=i+TEAM
        champ = champList[i]
        myS=0
        myC=0
        for j in range(TEAM):
            champ2 = champList[j+TEAM]
            if champ != champ2:
                myS += synerge[champ][champ2]["WinRate"]
            champ2 = champList[j]
            myC += counter[champ][champ2]["WinRate"]
        if TEAM>1:
            myS = myS/(TEAM-1)
        myC = myC/TEAM
        InputList.append(winRate[champ]['WinRate'])
        InputList.append(myS-50)
        InputList.append(myC-50)
    ret = np.array(InputList).reshape((1,TEAM*6))
    return ret

In [45]:
Arr = ['Aatrox','Camille',"Lulu",'Ornn','Teemo','Sett','Jhin','Kalista','Yasuo',"Yone"]
I = Convert(Arr)
mymodel.predict(I)

ValueError: Expected object or value

In [ ]:
'''
import json
with open('./Data/WSC/Synerge'+(str)(TEAM)+'.json','w') as f:
    json.dump(synerge,f,indent=4)
    f.close()
with open('./Data/WSC/Counter'+(str)(TEAM)+'.json','w') as f:
    json.dump(counter,f,indent=4)
    f.close()
with open('./Data/WSC/WinRate'+(str)(TEAM)+'.json','w') as f:
    json.dump(winRate,f,indent=4)
    f.close()
print("end")
'''